In [ ]:
!rm colab.zip
!rm -rf colab __MACOSX/
!gdown 1KaiWg6ulE-u0NU8IQelMKl1iCShJO9rZ
!unzip colab.zip -x "__MACOSX/*" > /dev/null

Downloading...
From (original): https://drive.google.com/uc?id=1KaiWg6ulE-u0NU8IQelMKl1iCShJO9rZ
From (redirected): https://drive.google.com/uc?id=1KaiWg6ulE-u0NU8IQelMKl1iCShJO9rZ&confirm=t&uuid=7af901d2-762f-491a-8602-05b8c6c406fe
To: /content/colab.zip
100% 585M/585M [00:04<00:00, 123MB/s]


In [ ]:
import os
import re
from tqdm.auto import tqdm
from datetime import datetime

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
def clear_text(t):
    return t.replace("\xa0", " ")

In [ ]:
SRC = 'colab/data/src'
DEST = 'colab/data/df'

os.makedirs(DEST, exist_ok=True)

## Находим все файлики

In [ ]:
files = set()

In [ ]:
for dir, _, files_list in os.walk(SRC):
    for file in files_list:
        if file.endswith('.html'):
            files.add(os.path.join(dir, file))

In [ ]:
len(files)

10182

# Парсим

In [ ]:
def parse(file):
    with open(file) as f:
        soup = BeautifulSoup(f.read(), 'html.parser')

        params = soup.find_all('div', class_=re.compile(r'CharacteristicsBlock__Row-.*'))

        row = {
            'id': file.split('/')[-1].removesuffix('.html'),
            'Название': soup.find('h1').get_text(),
            'Класс энергетической эффективности': soup.find(class_=re.compile(r'NewBuildingCard__Title.*')).get_text().split(': ')[-1]
        }

        value = soup.find(class_=re.compile(r'HouseInfo__Container.*')).find(class_=re.compile(r'Row__Value.*')).get_text()
        row['Ввод в эксплуатацию'] = clear_text(value)

        for param in params:
            param = param.find_all(class_=re.compile(r'CharacteristicsBlock__RowSpan.*'))
            row[clear_text(param[0].get_text())] = clear_text(param[1].get_text())

        params = soup.find_all('div', class_=re.compile(r'Index__Wrapper-.*'))

        for param in params:
            key = param.find_all(class_=re.compile(r'Index__Title.*'))[0].get_text()
            value = param.find_all(class_=re.compile(r'CircleValue__Value.*'))[0].get_text()
            row[key] = clear_text(value)

        return row

In [ ]:
LIMIT = 10000

data = []

for file in tqdm(list(files)[:LIMIT]):
    try:
        data.append(parse(file))
    except:
        print(f'error while parsing file: {file}')

  0%|          | 0/10000 [00:00<?, ?it/s]

error while parsing file: colab/data/src/51000/51865.html
error while parsing file: colab/data/src/46000/46077.html
error while parsing file: colab/data/src/50000/50325.html
error while parsing file: colab/data/src/46000/46986.html
error while parsing file: colab/data/src/40000/40315.html
error while parsing file: colab/data/src/53000/53432.html
error while parsing file: colab/data/src/50000/50324.html
error while parsing file: colab/data/src/54000/54867.html
error while parsing file: colab/data/src/46000/46508.html
error while parsing file: colab/data/src/39000/39535.html
error while parsing file: colab/data/src/48000/48340.html
error while parsing file: colab/data/src/55000/55186.html
error while parsing file: colab/data/src/34000/34699.html
error while parsing file: colab/data/src/41000/41215.html
error while parsing file: colab/data/src/49000/49379.html
error while parsing file: colab/data/src/55000/55206.html
error while parsing file: colab/data/src/47000/47738.html
error while pa

## Создаем табличку

In [ ]:
df = pd.DataFrame(data)
df.set_index('id', inplace=True)
df

,Название,Класс энергетической эффективности,Ввод в эксплуатацию,Класс недвижимости,Материал стен,Тип отделки,Свободная планировка,Количество этажей,Количество квартир,"Жилая площадь, м²",...,Индекс доступности инфраструктуры,Количество спортивныхплощадок,Наличие понижающихплощадок,Доступность инфраструктуры,Транспортная доступность,Индекс транспортной доступности,Зеленые зоны,Качество воздуха,Индекс зеленой зоны,Индекс качества воздуха
id,,,,,,,,,,,,,,,,,,,,,
50878,"п. Металлплощадка, б-р Строителей, д. 71, корп...",B,IV кв. 2025,Типовой,Кирпич,1 вариант,Есть,4,50,2 223,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41996,"ЖК ""Челны Сити""",Не нормируется,III кв. 2024,Комфорт,Монолит-кирпич,Нет,Нет,8,Нет,Нет,...,NaN,1,Есть,9,NaN,NaN,NaN,NaN,NaN,NaN
45490,DOGMA,B,II кв. 2026,Комфорт,Монолит-кирпич,1 вариант,Нет,17,856,35 071,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50119,"ЖК ""DOLCE VITA""",A,IV кв. 2025,Комфорт,Монолит-кирпич,Нет,Есть,2,Нет,Нет,...,NaN,1,Есть,7,NaN,NaN,NaN,NaN,NaN,NaN
40642,ЖК Городские сады,B,IV кв. 2024,Комфорт,Монолит-кирпич,1 вариант,Нет,20,387,23 290,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55922,г. Бердск,B,I кв. 2025,Комфорт,Кирпич,1 вариант,Нет,2,1,68,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50011,ЖК Сармат,B,IV кв. 2024,Комфорт,Монолит-кирпич,1 вариант,Нет,2,Нет,Нет,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49671,"Клубный поселок""Твоя Привилегия""",A,II кв. 2024,Типовой,Кирпич,1 вариант,Нет,5,208,12 206,...,NaN,1,Есть,4,NaN,NaN,NaN,NaN,NaN,NaN


## Смотрим на значения

In [ ]:
for col in df:
  if col == 'Название':
    continue

  print(col, df[col].unique())
  print()

Класс энергетической эффективности ['B' 'Не нормируется' 'A' 'D' 'C' 'A+' 'A++' 'G' 'E']

Ввод в эксплуатацию ['IV кв. 2025' 'III кв. 2024' 'II кв. 2026' 'IV кв. 2024' 'I кв. 2025'
 'II кв. 2024' 'II кв. 2025' 'IV кв. 2026' 'III кв. 2026' 'I кв. 2026'
 'II кв. 2030' 'I кв. 2027' 'I кв. 2024' 'III кв. 2025' 'IV кв. 2027'
 'IV кв. 2028' 'III кв. 2027' 'II кв. 2029' 'II кв. 2027' 'I кв. 2028'
 'I кв. 2031' 'IV кв. 2023' 'II кв. 2028' 'IV кв. 2033' 'III кв. 2028'
 'IV кв. 2031' 'III кв. 2023' 'IV кв. 2036' 'III кв. 2030' 'IV кв. 2022'
 'III кв. 2029' 'IV кв. 2029' 'IV кв. 2030' 'II кв. 2032' 'IV кв. 2032'
 'I кв. 2029' 'I кв. 2032' 'II кв. 2021' 'III кв. 2036' 'IV кв. 2021'
 'II кв. 2031' 'I кв. 2030' 'II кв. 2033' 'IV кв. 2018']

Класс недвижимости ['Типовой' 'Комфорт' 'Бизнес' 'Элитный' 'Другое']

Материал стен ['Кирпич' 'Монолит-кирпич' 'Другое' 'Монолит' 'Блоки' 'Панель']

Тип отделки ['1 вариант' 'Нет' '2 варианта' '3 варианта']

Свободная планировка ['Есть' 'Нет']

Количество этажей 

Веселые тут данные конечно. Развлечетесь их чистить :)

### Да/нет

для примера

In [ ]:
df['Наличие пандуса'] = df['Наличие пандуса'].map({'Есть': True, 'Нет': False})

### Итоговые типы

In [ ]:
df = df.convert_dtypes()
df.dtypes

Название                                         string[python]
Класс недвижимости                               string[python]
Материал стен                                    string[python]
Тип отделки                                      string[python]
Свободная планировка                             string[python]
Количество этажей                                string[python]
Количество квартир                               string[python]
Жилая площадь, м²                                string[python]
Высота потолков, м                               string[python]
Велосипедные дорожки                             string[python]
Количество детских площадок                      string[python]
Количество спортивных площадок                   string[python]
Количество площадок для сбора мусора             string[python]
Количество мест в паркинге                       string[python]
Гостевые места на придомовой территории          string[python]
Гостевые места вне придомовой территории

## Сохраним датафрейм

In [ ]:
df.to_csv(os.path.join(DEST, f'{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}_home_ru_{df.shape[0]}.csv'))